<a href="https://colab.research.google.com/github/LeninGF/CoursesNotes/blob/main/InteligenciaArtificalGenerativa/Problems/transformersII-huggingface/Ejercicios-IAG-2024B-LeninFalconi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejericios Transformers con HuggingFace

Coder: Lenin Falconí

Fecha: 2024-12-16



## AutoModel

Se explora el uso de AutoModel que es una clase que no provee de salida de clasificación y que por tanto puede ser utilizada para crear modelos customizados

In [ ]:
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
text = "I am an example sequence for text classification"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Se define la Classification Head

In [ ]:
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleClassifier, self).__init__()
        self.fc = nn.Linear(input_size, num_classes)
    def forward(self, x):
        return self.fc(x)

En la Aplicación se realiza los siguientes pasos para observar la forma de los tensores de salida

In [ ]:
text = "afjsdalkfjlasdfk525 52523faklsdlfkdsa  5 a fjfsdajfladsjf"

In [ ]:
 import torch
 inputs = tokenizer(
      text, return_tensors="pt", padding=True,
     truncation=True, max_length=64
      )
 outputs = model(**inputs)
 pooled_output = outputs.pooler_output
 print("Hidden states size: ", outputs.last_hidden_state.shape)
 print("Pooled output size: ", pooled_output.shape)
 classifier_head = SimpleClassifier(
 pooled_output.size(-1), num_classes=2
 )
 logits = classifier_head(pooled_output)
 probs = torch.softmax(logits, dim=1)
 print("Predicted Class Probabilities: ", probs)


Hidden states size:  torch.Size([1, 33, 768])
Pooled output size:  torch.Size([1, 768])
Predicted Class Probabilities:  tensor([[0.5271, 0.4729]], grad_fn=<SoftmaxBackward0>)


In [ ]:
probs.sum()

tensor(1.0000, grad_fn=<SumBackward0>)

Si lo anterior es cierto y se puede personalizar la clasificación a un dataset específico hagamos una prueba de entrenamiento.

In [ ]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# prompt: considering the code before provide a code to train the model on a classification dataset

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModel, AutoTokenizer
import numpy as np

from datasets import load_dataset

ds = load_dataset("stanfordnlp/imdb")

# Tokenization and Data Loading
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

ds

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
# Number of samples to use for training and validation
num_train_samples = 5000  # Example: using 5000 samples for training
num_val_samples = 1000  # Example: using 1000 samples for validation

# Select a subset of the dataset for training and validation
train_ds = ds['train'].select(range(num_train_samples))
val_ds = ds['test'].select(range(num_val_samples))

In [ ]:
def tokenize_data(texts):
    encoded_input = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=64)
    return encoded_input


train_encodings = tokenize_data(train_ds['text'])
y_train = np.array(train_ds['label'])
y_val = np.array(val_ds['label'])
val_encodings = tokenize_data(val_ds['text'])

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(y_train))
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(y_val))

train_loader = DataLoader(train_dataset, batch_size=125, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=125)


# Model and Training Setup
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleClassifier, self).__init__()
        self.fc = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.fc(x)

classifier_head = SimpleClassifier(768, 2) # Assuming BERT's output size is 768

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
classifier_head.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(list(model.parameters()) + list(classifier_head.parameters()), lr=5e-5)

# Training Loop
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    val_correct_predictions = 0
    val_total_samples = 0
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = classifier_head(pooled_output)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, predicted_labels = torch.max(logits, 1)
        correct_predictions += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_samples
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.4f}")


    # Validation (example)
    model.eval() # Set the model to evaluation mode
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            pooled_output = outputs.pooler_output
            logits = classifier_head(pooled_output)

            #Calculate validation accuracy or other metrics here
            _, predicted_labels = torch.max(logits, 1)
            val_correct_predictions += (predicted_labels == labels).sum().item()
            val_total_samples += labels.size(0)

        val_accuracy = val_correct_predictions / val_total_samples
        print(f"Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_accuracy:.4f}")


print("Training Complete!")

Epoch 1/5, Train Loss: 0.0405, Train Accuracy: 1.0000
Epoch 1/5, Validation Accuracy: 1.0000
Epoch 2/5, Train Loss: 0.0016, Train Accuracy: 1.0000
Epoch 2/5, Validation Accuracy: 1.0000
Epoch 3/5, Train Loss: 0.0005, Train Accuracy: 1.0000
Epoch 3/5, Validation Accuracy: 1.0000
Epoch 4/5, Train Loss: 0.0002, Train Accuracy: 1.0000
Epoch 4/5, Validation Accuracy: 1.0000
Epoch 5/5, Train Loss: 0.0001, Train Accuracy: 1.0000
Epoch 5/5, Validation Accuracy: 1.0000
Training Complete!


## Clasificación de Texto con AutoModelForSequenceClassification



In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
text = ["Service was incredible but taste was terrible.",
        "Music was awesome in the bar and the food was delicious."]
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)
# getting logits
logits = outputs.logits
# getting probabilities out of logits
probs = torch.softmax(logits, dim=1)
# getting the predicted class index
predicted_class_logits = torch.argmax(logits, dim=1)

# Get the predicted class index (from probabilities - should be the same)
predicted_class_probs = torch.argmax(probs, dim=1)

# Print the results
print(f"Logits: {logits}")
print(f"Probabilities: {probs}")
print(f"Predicted class index (from logits): {predicted_class_logits + 1} star.")
print(f"Predicted class index (from probabilities): {predicted_class_probs + 1} star.")

Logits: tensor([[ 0.7153,  1.2554,  0.9619, -0.5681, -1.8381],
        [-2.5750, -2.3530, -0.2007,  1.7969,  2.6795]],
       grad_fn=<AddmmBackward0>)
Probabilities: tensor([[0.2298, 0.3945, 0.2941, 0.0637, 0.0179],
        [0.0035, 0.0044, 0.0379, 0.2792, 0.6750]], grad_fn=<SoftmaxBackward0>)
Predicted class index (from logits): tensor([2, 5]) star.
Predicted class index (from probabilities): tensor([2, 5]) star.


## AutoModelForCausalLM

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
prompt = "Theory of relativity explains gravity with geometry and"
inputs = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(inputs, max_length=25)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text:
Theory of relativity explains gravity with geometry and physics.

Theory of relativity explains gravity with geometry and physics.



## Resumen de Texto


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
from datasets import load_dataset

dataset = load_dataset("ILSUM/ILSUM-1.0", "English")
print(f"Features: {dataset['train'].column_names}")

README.md:   0%|          | 0.00/4.78k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/46.5M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

val.csv:   0%|          | 0.00/3.37M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12565 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4487 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/898 [00:00<?, ? examples/s]

Features: ['id', 'Article', 'Heading', 'Summary']


In [9]:
example = dataset["train"][21]
example["Article"]

'This is how an Apple Watch saved a man\'s life after detecting accidentIt all started when Gabe Burdett was waiting for his father Bob at their pre-designated location for some mountain biking at the Riverside State Park when he received a text alert from his dad\'s Apple Watch, saying it had detected a "hard fall".Burdett, from city of Spokane in Washington State later received another update from the Watch, saying his father had reached Sacred Heart Medical Center."We drove straight there but he was gone when we arrived. I get another update from the Watch saying his location has changed with a map location of SHMC. Dad flipped his bike at the bottom of Doomsday, hit his head and was knocked out until sometime during the ambulance ride," Burdett wrote in a Facebook post.The Watch notified 911 with the location and within 30 minutes, emergency medical services (EMS) took the injured Bob to the hospital."If you own an Apple Watch, set up your hard fall detection, it\'s not just for wh

In [10]:
example["Summary"]

'“Dad flipped his bike at the bottom of Doomsday, hit his head and was knocked out until sometime during the ambulance ride. The watch had called 911 with his location and EMS had him scooped up and to the hospital in under a 1/2 hr,” says Gabe in his post.'

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [12]:
input_ids = tokenizer.encode("summarize: " + example["Article"],
return_tensors="pt",
max_length=512,
truncation=True
 )

In [13]:
summary_ids = model.generate(input_ids, max_length=150)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [14]:
print("Original Text:")
print(example["Article"])
print("\nGenerated Summary:")
print(summary)

Original Text:
This is how an Apple Watch saved a man's life after detecting accidentIt all started when Gabe Burdett was waiting for his father Bob at their pre-designated location for some mountain biking at the Riverside State Park when he received a text alert from his dad's Apple Watch, saying it had detected a "hard fall".Burdett, from city of Spokane in Washington State later received another update from the Watch, saying his father had reached Sacred Heart Medical Center."We drove straight there but he was gone when we arrived. I get another update from the Watch saying his location has changed with a map location of SHMC. Dad flipped his bike at the bottom of Doomsday, hit his head and was knocked out until sometime during the ambulance ride," Burdett wrote in a Facebook post.The Watch notified 911 with the location and within 30 minutes, emergency medical services (EMS) took the injured Bob to the hospital."If you own an Apple Watch, set up your hard fall detection, it's not 

## Traducción Automática

In [1]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
dataset = load_dataset("techiaith/legislation-gov-uk_en-cy")
idx = 0
dataset["train"]['source'][idx]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/64726 [00:00<?, ? examples/s]

'2 Regulations under section 1: supplementary'

In [3]:
dataset["train"]['target'][idx]

'2 Rheoliadau o dan adran 1: atodol'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "Helsinki-NLP/opus-mt-en-es"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

english_inputs = ["Hello, it was raining when I arrived today.", "what is your name?", "There was a time when Green Leaves was a popular folk song"]
for english_input in english_inputs:
  input_ids = tokenizer(english_input, return_tensors="pt")
  translated_ids = model.generate(**input_ids)
  translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
  print(f"English: {english_input}: Spanish: {translated_text}")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

English: Hello, it was raining when I arrived today.: Spanish: Hola, estaba lloviendo cuando llegué hoy.
English: what is your name?: Spanish: ¿Cómo te llamas?
English: There was a time when Green Leaves was a popular folk song: Spanish: Hubo un tiempo en que Green Leaves era una canción popular


# Ejercicios




## Ejercicio de Resumen de texto
Resumir una opinión de productoEn este ejercicio de resumen de texto,
examinaremos diferentes aspectos de un dataset que contiene reseñas de
clientes, además de mostrar un ejemplo de secuencia de entrada y el resumen
generado.

### Instrucciones
1.  Mostrar los nombres de las características (columnas) en los datos, accediendo a la
división 'train' del dataset descargado.
2.  Utilizar las variables y métodos necesarios para codificar el ejemplo de entrada,
pásalo al modelo para generar un resumen y decodificar el resumen.

In [15]:
from datasets import load_dataset

dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)
idx = 99
print(dataset["full"][idx])


{'rating': 4.0, 'title': 'Easy to Use!', 'text': "My husband started a business recently and was looking at options for his bookkeeping. He knew that he wanted to use QuickBooks from his previous experience. Originally he was going to use the web based version, but since this comes with a year of enhanced payroll, he decided to go with this. He consulted with an accountant in getting it set up for his company's needs. He did have to contact customer support to help him get the enhanced payroll feature working. For some reason, it had the number 1234567 prefilled and he couldn't change it without customer support. So far it has been easy to use and has done everything he's needed it to do.", 'images': [], 'asin': 'B07FZ5HZLM', 'parent_asin': 'B07FZ5HZLM', 'user_id': 'AHV6QCNBJNSGLATP56JAWJ3C4G2A', 'timestamp': 1548092773968, 'helpful_vote': 0, 'verified_purchase': False}


In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [17]:
input_ids = tokenizer.encode("summarize: " +dataset['full'][idx]["text"],
return_tensors="pt",
max_length=512,
truncation=True
 )

In [18]:
summary_ids = model.generate(input_ids, max_length=150)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [19]:
print("Original Text:")
print(dataset['full'][idx]["text"])
print("\nGenerated Summary:")
print(summary)

Original Text:
My husband started a business recently and was looking at options for his bookkeeping. He knew that he wanted to use QuickBooks from his previous experience. Originally he was going to use the web based version, but since this comes with a year of enhanced payroll, he decided to go with this. He consulted with an accountant in getting it set up for his company's needs. He did have to contact customer support to help him get the enhanced payroll feature working. For some reason, it had the number 1234567 prefilled and he couldn't change it without customer support. So far it has been easy to use and has done everything he's needed it to do.

Generated Summary:
he started a business and was looking at options for his bookkeeping. he decided to use the web based version of QuickBooks. he had to contact customer support to get the enhanced payroll feature working.


## Ejercicio Traducción

La misión del libro de frases en español

Usted es redactor de contenido en una editorial de guías de viaje de renombre. El próximo título
a publicarse es una guía de viajes por España para hablantes de inglés. Sin embargo, debido a la
alta demanda y a los recursos humanos limitados, le han asignado la urgente tarea de redactar
una página del "libro de frases en español", cubriendo palabras y frases esenciales de
supervivencia en español.

¡Por suerte, los LLMs están aquí para ayudar! En este ejercicio, intentará usar un modelo LLM preentrenado para realizar traducciones del inglés al español y comenzará esta misión
importante traduciendo las primeras cinco frases comunes en inglés al español

Utilizar las clases y métodos específicos de la tarea para cargar el tokenizer y el modelo.

Completar las instrucciones para codificar las secuencias de entrada, generar las traducciones y
decodificarlas.

- Para las codificaciones, usar un argumento adicional para devolverlas como tensores de PyTorch

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "Helsinki-NLP/opus-mt-en-es"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

english_inputs = ["Hello, my name is John and I am not a terrorist", "where is the bus station to El Campanario Hotel?", "Where can I change my money to Euros? Sorry for Brexit", "The thing about Brexit is our love about imperialism"]
for english_input in english_inputs:
  input_ids = tokenizer(english_input, return_tensors="pt")
  translated_ids = model.generate(**input_ids)
  translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
  print(f"English: {english_input}: Spanish: {translated_text}")

English: Hello, my name is John and I am not a terrorist: Spanish: Hola, mi nombre es John y no soy un terrorista.
English: where is the bus station to El Campanario Hotel?: Spanish: ¿Dónde está la estación de autobuses al Hotel El Campanario?
English: Where can I change my money to Euros? Sorry for Brexit: Spanish: ¿Dónde puedo cambiar mi dinero a euros?
English: The thing about Brexit is our love about imperialism: Spanish: Lo que pasa con el Brexit es nuestro amor por el imperialismo
